## Sellar MDO

In [1]:
from engine.torchengine import AnalyticalSet, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals, ElimResidual, ParallelResiduals
from engine.torchdata import generate_optim_functions
from scipy.optimize import minimize
import numpy as np
import torch
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
varorder = ['x1','x2','x3','u1','u2']
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}

sellarobj = (('x2', 'x3', 'u1', 'u2'), lambda x2, x3, u1, u2: x3**2 + x2 + u1 + torch.exp(-u2))
sellar1 = (('x1', 'x2', 'x3', 'u2'), ('u1',), lambda x1, x2, x3, u2: x1**2 + x2 + x3 - 0.2*u2)
sellar2 = (('x1', 'x2', 'u1'), ('u2',), lambda x1, x2, u1: u1**0.5 + x1 + x2)
ineqcon1 = (('u1',), lambda u1: 1-u1/3.16)
ineqcon2 = (('u2',), lambda u2: u2/24-1)

In [3]:
set1 = AnalyticalSet(sellar1, indices)
set2 = AnalyticalSet(sellar2, indices)
con1 = Function(ineqcon1, indices)
con2 = Function(ineqcon2, indices)
obj = Function(sellarobj, indices)

coupled = EliminateAnalysisMergeResiduals(functions=[set1.residual, set2.residual])
solver = ElimResidual(coupled, ['u1','u2'], indices)
ineqcons = EliminateAnalysisMergeResiduals([], [con1, con2])

In [4]:
# Disciplines
# [{1},{2},{3,4,5},{6,7}], obj, eq, ineq
# -> variant one: A=[1,2], B=[3,4,5,6,7] # 2 options: reduced AAO / reduced IDF
# ## reduced AAO
# neweq = EliminateAnalysisMergeResiduals([], [3.r,4.r,5.r,6.r,7.r,eq]) 
# ## reduced IDF
# idf_eq = Parallel([3.a,4.a,5.a,6.a,7.a])
# neweq = EliminateAnalysisMergeResiduals([], [idf_eq, eq]) 
# ##BOTH:
# objf, eqf, ineqf = EliminateAnalysisMergeResiduals([1.a,2.a], [obj, neweq, ineq])
# -> variant two: [1,2, ([3,4],5), (6,7)] # reduced MDF (tearing based)
# block1 = EliminateAnalysisMergeResiduals([3.a,4.a], [5.r])
# block2 = EliminateAnalysisMergeResiduals([], [6.r, 7.r])
# objf, eqf, ineqf = EliminateAnalysisMergeResiduals([1,2, block1, block2], [ obj, eq, ineq])
# -> variant three: (1,2,3,4,5,6,7) # baseline MDF
# objf, eqf, ineqf = EliminateAnalysisMergeResiduals([1,2, 3, 4, 5, 6, 7], [ obj, eq, ineq])

### MDA

In [5]:
x0 = torch.tensor([2.,0.,0.,0.1,0.1], dtype=torch.float64) # Important for finite differences
sol = solver(x0)
sol.numpy(), coupled(sol).numpy()

(array([2, 0, 0, 3.24, 3.8]), array([-0, 0]))

In [6]:
x0 = torch.tensor([0.5,.5,0.5,0.1,0.1], dtype=torch.float64)

### MDF

In [7]:
P = EliminateAnalysis([solver], [obj, ineqcons])
objective_idx, inequality_idx, equality_idx = 0,1,None

### IDF

In [8]:
P = ParallelResiduals([set1.analysis, set2.analysis], [obj, ineqcons])
objective_idx, inequality_idx, equality_idx = 0,1,2

### AAO

In [9]:
P = EliminateAnalysis(functions=[obj, ineqcons, coupled])
objective_idx, inequality_idx, equality_idx = 0,1,2

### Solve

In [10]:
optim_indices=P.structure[0]
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, _ = generate_optim_functions(P, optim_indices, x0, inequality_direction='positive-null', objective=objective_idx, inequalities=inequality_idx, equalities=equality_idx)

In [11]:
bnds = [(0, 10), (0, 10), (0, 10), (3.16, None), (None, 24)]
bnds_problem = [bnds[elt] for elt in optim_indices]
constraints = [{'type': 'ineq', 'fun': ineq_function, 'jac':dineq}]
constraints = [{'type': 'eq', 'fun': eq_function, 'jac': deq}]
# Solve the optimization problem
minimize(obj_function, xguess, jac=dobj, bounds=bnds_problem, constraints=constraints, method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.1833939516406082
       x: [ 9.330e-18  1.464e-16  3.160e+00  3.755e+00  1.978e+00]
     nit: 6
     jac: [ 1.000e+00  1.020e-16  1.000e+00 -2.339e-02  0.000e+00]
    nfev: 7
    njev: 6

In [12]:
obj(x0).item(), ineqcons(x0).detach().numpy(), x0.detach().numpy()

(3.1833939516406082, array([-0, -0.84]), array([1.98, 0, 0, 3.16, 3.76]))

In [61]:
# Checking derivatives
# delta = 1e-7
# (obj_function(xguess+np.eye(3)[2]*delta)-obj_function(xguess))/delta